# Code details

## Summary
This notebook contains the main class that are composing our neural networkl 
- #### The Model 
Class that setup the shape of the model of our Convolutional Neural Network(CNN) 
- #### The Loader
Class that take charge of the pre-processing of the training and testing datasets
- #### The Main 
Function that initialize multiple parameters: image sizes and paths,number of epoch and batch size

## CNN Model

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
from tensorflow import keras

import tensorflow as tf
import datetime


class CnnModel:
    def __init__(self, fileNameNeural, inputShape, classNumber, active_log=False):
        self.fileNameNeural = fileNameNeural
        self.inputShape = inputShape
        self.classNumber = classNumber
        self.active_log = active_log 
        # set cnn model            
        self.model = keras.Sequential([
        keras.layers.Conv2D(32, 3, activation='relu', input_shape=self.inputShape),
        keras.layers.MaxPooling2D(2),
        keras.layers.Dropout(0.2),
        keras.layers.Conv2D(64, 3, activation='relu'),
        keras.layers.MaxPooling2D(2),
        keras.layers.Dropout(0.2),
        keras.layers.Conv2D(64, 3, activation='relu'),
        keras.layers.Flatten(),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(self.classNumber)
        ])
        
   # compile model
        self.model.compile(optimizer='adam',
                           loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                           metrics=['accuracy'])
        if self.active_log:
            # complete logs
            self.log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
            self.tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=self.log_dir, histogram_freq=1)
            
        # compile model
        self.model.compile(optimizer='adam',
                           loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                           metrics=['accuracy'])
        if self.active_log:
            # complete logs
            self.log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
            self.tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=self.log_dir, histogram_freq=1)

A convolutionnal neural network is composed of 2 types of layers : 
- convolutional layers, that contains convolution , maxPooling and dropout functions
`                
            keras.layers.Conv2D(32, 3, activation='relu', input_shape=self.inputShape),
            keras.layers.MaxPooling2D(2),
            keras.layers.Dropout(0.2),
`
- a classic neural network that start with a flatten of the previous data and end with the class of the dataset 
`
            keras.layers.Flatten(),
            keras.layers.Dense(64, activation='relu'),
            keras.layers.Dense(self.classNumber)
`


In [2]:
class CnnModel(CnnModel):
    def train(self, train_data, train_label, test_data, test_label, epochNumber):
        # train model
        if self.active_log:
            self.model.fit(train_data, train_label, validation_data=(test_data, test_label), epochs=epochNumber, callbacks=[self.tensorboard_callback])
        else:
            self.model.fit(train_data, train_label, validation_data=(test_data, test_label), epochs=epochNumber)

    def load(self, fileName):
        # load model
        self.model = keras.models.load_model(fileName + '.h5')
        # compile model
        self.model.compile(optimizer='adam',
                               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                               metrics=['accuracy'])
        
    def save(self):
        # save initialize neural network
        self.model.save(self.fileNameNeural + '.h5')

    def evaluate(self, test_images, test_labels):
        # evaluate model
        test_loss, test_acc = self.model.evaluate(test_images, test_labels, verbose=3)
        print('\nTest accuracy:', test_acc)
        print('\nLoss: ', test_loss)


##  Loader

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import pathlib
import numpy as np
import matplotlib.pyplot as plt
import os


class Loader:
    def __init__(self, pathToData, batchSize, height, width, className):
        # loading parameter
        self.BATCH_SIZE = batchSize
        self.IMG_HEIGHT = height
        self.IMG_WIDTH = width
        # load data dir
        self.data_dir = pathlib.Path(pathToData)
        # display class name
        self.CLASS_NAMES = className
        self.list_ds = tf.data.Dataset.list_files(str(self.data_dir/'*/*'))

    def print_list_ds_n(self, n):
        for f in self.list_ds.take(n):
            print(f)

    def show_batch(self, image_batch, label_batch, n):
        plt.figure(figsize=(10, 10))
        for n in range(n):
            ax = plt.subplot(5, 5, n + 1)
            plt.imshow(image_batch[n])
            plt.title(self.CLASS_NAMES[label_batch[n] == 1][0].title())
            plt.axis('off')
        plt.show()

    def get_label(self, file_path):
        parts = tf.strings.split(file_path, os.path.sep)
        return tf.equal(self.CLASS_NAMES, parts[-2])

    def decode_img(self, img):
        # channels = 3 for rgb image channels for grayscale
        img = tf.image.decode_jpeg(img, channels=1)
        img = tf.image.convert_image_dtype(img, tf.float32)
        return tf.image.resize(img, [self.IMG_WIDTH, self.IMG_HEIGHT])

    def process_path(self, file_path):
        label = self.get_label(file_path)
        img = tf.io.read_file(file_path)
        img = self.decode_img(img)
        return img, label

    def prepare_for_training(self, ds, cache=True, shuffle_buffer_size=40):
        if cache:
            if isinstance(cache, str):
                ds = ds.cache(cache)
            else:
                ds = ds.cache()
        ds = ds.shuffle(buffer_size=shuffle_buffer_size)
        # Repeat forever
        ds = ds.repeat()
        ds = ds.batch(self.BATCH_SIZE)
        # `prefetch` lets the dataset fetch batches in the background while the model
        # is training.
        ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

        return ds

    def resize_label(self, data):
        # resize test labels
        x = []
        for item in data:
            for index in range(len(item)):
                if item[index]:
                    x.append(index)

        return np.array(x)

    def load_data(self):
        # Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
        labeled_ds = self.list_ds.map(self.process_path, num_parallel_calls=tf.data.experimental.AUTOTUNE)
        train_ds = self.prepare_for_training(labeled_ds, cache=False)
        image_batch, label_batch = next(iter(train_ds))
        print("image batch:", type(image_batch.numpy()))
        print("label batch:", type(label_batch.numpy()))

        """
        # See data as ndarray
        show_batch(image_batch.numpy(), label_batch.numpy())
        """
        return image_batch.numpy(), self.resize_label(label_batch.numpy())


### Main function

In [ ]:

if __name__ == "__main__":
    fileName = 'neural_network'
    DATA_PATH_TRAIN = 'resource/data/train'
    DATA_PATH_TEST = 'resource/data/test'
    BATCH_SIZE = 50
    IMG_HEIGHT = 224
    IMG_WIDTH = 224
    inputShape = (IMG_WIDTH, IMG_HEIGHT, 1)
    EPOCHS = 1000
    CLASS_NAME = ['NORMAL', 'BACTERIA', 'VIRUS']

    # init model
    model = CnnModel(fileName, inputShape, 3, False)

    # replace the actual model with an existing one from h5 format file
    # model.load('neural_network')

    # load train data
    loader_train = Loader(DATA_PATH_TRAIN, BATCH_SIZE, IMG_HEIGHT, IMG_WIDTH, CLASS_NAME)
    train_data, train_label = loader_train.load_data()
    train_data = train_data.reshape((BATCH_SIZE, IMG_WIDTH, IMG_HEIGHT, 1))

    # load test data
    loader_test = Loader(DATA_PATH_TEST, BATCH_SIZE, IMG_HEIGHT, IMG_WIDTH, CLASS_NAME)
    test_data, test_label = loader_test.load_data()
    test_data = test_data.reshape((BATCH_SIZE, IMG_WIDTH, IMG_HEIGHT, 1))

    # train model then evaluate with test data
    model.train(train_data, train_label, test_data, test_label, EPOCHS)
    # save model in a .h5 file
    model.save()


image batch: <class 'numpy.ndarray'>
label batch: <class 'numpy.ndarray'>
image batch: <class 'numpy.ndarray'>
label batch: <class 'numpy.ndarray'>
Train on 50 samples, validate on 50 samples
Epoch 1/1000
50/50 [==============================] - 3s 53ms/sample - loss: 2.9275 - accuracy: 0.3800 - val_loss: 4.9796 - val_accuracy: 0.3800
Epoch 2/1000
50/50 [==============================] - 2s 45ms/sample - loss: 4.9949 - accuracy: 0.4000 - val_loss: 1.6803 - val_accuracy: 0.0800
Epoch 3/1000
50/50 [==============================] - 2s 47ms/sample - loss: 1.4875 - accuracy: 0.3200 - val_loss: 1.0230 - val_accuracy: 0.5200
Epoch 4/1000
50/50 [==============================] - 2s 47ms/sample - loss: 1.1208 - accuracy: 0.3800 - val_loss: 1.0347 - val_accuracy: 0.5200
Epoch 5/1000
50/50 [==============================] - 2s 46ms/sample - loss: 1.0927 - accuracy: 0.3800 - val_loss: 1.1341 - val_accuracy: 0.0800
Epoch 6/1000
50/50 [==============================] - 2s 46ms/sample - loss: 1.0948

50/50 [==============================] - 2s 47ms/sample - loss: 6.1567e-05 - accuracy: 1.0000 - val_loss: 1.7949 - val_accuracy: 0.6800
Epoch 56/1000
50/50 [==============================] - 2s 46ms/sample - loss: 6.0199e-05 - accuracy: 1.0000 - val_loss: 1.8028 - val_accuracy: 0.6600
Epoch 57/1000
50/50 [==============================] - 2s 46ms/sample - loss: 6.8237e-05 - accuracy: 1.0000 - val_loss: 1.8192 - val_accuracy: 0.6600
Epoch 58/1000
50/50 [==============================] - 2s 46ms/sample - loss: 5.2823e-05 - accuracy: 1.0000 - val_loss: 1.8395 - val_accuracy: 0.6600
Epoch 59/1000
50/50 [==============================] - 2s 46ms/sample - loss: 5.0163e-05 - accuracy: 1.0000 - val_loss: 1.8606 - val_accuracy: 0.6600
Epoch 60/1000
50/50 [==============================] - 2s 46ms/sample - loss: 2.4103e-05 - accuracy: 1.0000 - val_loss: 1.8781 - val_accuracy: 0.6600
Epoch 61/1000
50/50 [==============================] - 2s 46ms/sample - loss: 3.0248e-05 - accuracy: 1.0000 - val_

50/50 [==============================] - 2s 47ms/sample - loss: 7.6626e-06 - accuracy: 1.0000 - val_loss: 2.1481 - val_accuracy: 0.6000
Epoch 110/1000
50/50 [==============================] - 2s 46ms/sample - loss: 8.0036e-06 - accuracy: 1.0000 - val_loss: 2.1416 - val_accuracy: 0.6200
Epoch 111/1000
50/50 [==============================] - 2s 47ms/sample - loss: 6.3037e-06 - accuracy: 1.0000 - val_loss: 2.1338 - val_accuracy: 0.6200
Epoch 112/1000
50/50 [==============================] - 2s 46ms/sample - loss: 1.0948e-05 - accuracy: 1.0000 - val_loss: 2.1332 - val_accuracy: 0.6200
Epoch 113/1000
50/50 [==============================] - 2s 47ms/sample - loss: 6.2989e-06 - accuracy: 1.0000 - val_loss: 2.1345 - val_accuracy: 0.6200
Epoch 114/1000
50/50 [==============================] - 2s 46ms/sample - loss: 7.6197e-06 - accuracy: 1.0000 - val_loss: 2.1411 - val_accuracy: 0.6200
Epoch 115/1000
50/50 [==============================] - 2s 46ms/sample - loss: 9.1145e-06 - accuracy: 1.0000 

50/50 [==============================] - 3s 50ms/sample - loss: 4.7635e-06 - accuracy: 1.0000 - val_loss: 2.2161 - val_accuracy: 0.6200
Epoch 164/1000
50/50 [==============================] - 2s 46ms/sample - loss: 2.5868e-06 - accuracy: 1.0000 - val_loss: 2.2129 - val_accuracy: 0.6200
Epoch 165/1000
50/50 [==============================] - 2s 47ms/sample - loss: 3.5286e-06 - accuracy: 1.0000 - val_loss: 2.2153 - val_accuracy: 0.6200
Epoch 166/1000
50/50 [==============================] - 2s 49ms/sample - loss: 3.6573e-06 - accuracy: 1.0000 - val_loss: 2.2238 - val_accuracy: 0.6200
Epoch 167/1000
50/50 [==============================] - 3s 50ms/sample - loss: 4.1532e-06 - accuracy: 1.0000 - val_loss: 2.2293 - val_accuracy: 0.6200
Epoch 168/1000
50/50 [==============================] - 2s 49ms/sample - loss: 2.5368e-06 - accuracy: 1.0000 - val_loss: 2.2364 - val_accuracy: 0.6200
Epoch 169/1000
50/50 [==============================] - 2s 47ms/sample - loss: 3.8767e-06 - accuracy: 1.0000 

50/50 [==============================] - 3s 52ms/sample - loss: 2.3246e-06 - accuracy: 1.0000 - val_loss: 2.3618 - val_accuracy: 0.6000
Epoch 218/1000
50/50 [==============================] - 2s 47ms/sample - loss: 1.6665e-06 - accuracy: 1.0000 - val_loss: 2.3687 - val_accuracy: 0.6000
Epoch 219/1000
50/50 [==============================] - 2s 47ms/sample - loss: 2.3103e-06 - accuracy: 1.0000 - val_loss: 2.3723 - val_accuracy: 0.6000
Epoch 220/1000
50/50 [==============================] - 2s 47ms/sample - loss: 2.7346e-06 - accuracy: 1.0000 - val_loss: 2.3786 - val_accuracy: 0.6000
Epoch 221/1000
50/50 [==============================] - 2s 47ms/sample - loss: 3.0422e-06 - accuracy: 1.0000 - val_loss: 2.3817 - val_accuracy: 0.6000
Epoch 222/1000
50/50 [==============================] - 2s 48ms/sample - loss: 1.5449e-06 - accuracy: 1.0000 - val_loss: 2.3793 - val_accuracy: 0.6000
Epoch 223/1000
50/50 [==============================] - 2s 47ms/sample - loss: 2.4080e-06 - accuracy: 1.0000 

...

50/50 [==============================] - 2s 46ms/sample - loss: 4.2915e-07 - accuracy: 1.0000 - val_loss: 2.6771 - val_accuracy: 0.6000
Epoch 488/1000
50/50 [==============================] - 2s 46ms/sample - loss: 5.2214e-07 - accuracy: 1.0000 - val_loss: 2.6909 - val_accuracy: 0.6000
Epoch 489/1000
50/50 [==============================] - 2s 46ms/sample - loss: 3.7193e-07 - accuracy: 1.0000 - val_loss: 2.6911 - val_accuracy: 0.6000
Epoch 490/1000
50/50 [==============================] - 2s 46ms/sample - loss: 3.8147e-07 - accuracy: 1.0000 - val_loss: 2.6905 - val_accuracy: 0.6000
Epoch 491/1000
50/50 [==============================] - 2s 46ms/sample - loss: 4.1485e-07 - accuracy: 1.0000 - val_loss: 2.6860 - val_accuracy: 0.6000
Epoch 492/1000
50/50 [==============================] - 2s 46ms/sample - loss: 6.3658e-07 - accuracy: 1.0000 - val_loss: 2.6655 - val_accuracy: 0.6000
Epoch 493/1000
50/50 [==============================] - 2s 46ms/sample - loss: 3.4809e-07 - accuracy: 1.0000 

50/50 [==============================] - 2s 46ms/sample - loss: 4.1723e-07 - accuracy: 1.0000 - val_loss: 2.6680 - val_accuracy: 0.6200
Epoch 542/1000
50/50 [==============================] - 2s 46ms/sample - loss: 3.0518e-07 - accuracy: 1.0000 - val_loss: 2.7132 - val_accuracy: 0.6200
Epoch 543/1000
50/50 [==============================] - 2s 46ms/sample - loss: 3.4809e-07 - accuracy: 1.0000 - val_loss: 2.7510 - val_accuracy: 0.6000
Epoch 544/1000
50/50 [==============================] - 2s 46ms/sample - loss: 1.8358e-07 - accuracy: 1.0000 - val_loss: 2.7755 - val_accuracy: 0.6000
Epoch 545/1000
50/50 [==============================] - 2s 46ms/sample - loss: 2.0266e-07 - accuracy: 1.0000 - val_loss: 2.7918 - val_accuracy: 0.6000
Epoch 546/1000
50/50 [==============================] - 2s 46ms/sample - loss: 3.0279e-07 - accuracy: 1.0000 - val_loss: 2.8083 - val_accuracy: 0.6000
Epoch 547/1000
50/50 [==============================] - 2s 46ms/sample - loss: 2.3842e-07 - accuracy: 1.0000 

50/50 [==============================] - 2s 46ms/sample - loss: 1.7881e-07 - accuracy: 1.0000 - val_loss: 2.8401 - val_accuracy: 0.6000
Epoch 596/1000
50/50 [==============================] - 2s 46ms/sample - loss: 7.8201e-07 - accuracy: 1.0000 - val_loss: 2.8264 - val_accuracy: 0.6000
Epoch 597/1000
50/50 [==============================] - 2s 46ms/sample - loss: 3.0041e-07 - accuracy: 1.0000 - val_loss: 2.7982 - val_accuracy: 0.6000
Epoch 598/1000
50/50 [==============================] - 2s 46ms/sample - loss: 2.9564e-07 - accuracy: 1.0000 - val_loss: 2.7497 - val_accuracy: 0.6200
Epoch 599/1000
50/50 [==============================] - 2s 46ms/sample - loss: 2.1458e-07 - accuracy: 1.0000 - val_loss: 2.7139 - val_accuracy: 0.6200
Epoch 600/1000
50/50 [==============================] - 2s 46ms/sample - loss: 4.0769e-07 - accuracy: 1.0000 - val_loss: 2.6682 - val_accuracy: 0.6200
Epoch 601/1000
50/50 [==============================] - 2s 47ms/sample - loss: 3.1710e-07 - accuracy: 1.0000 

50/50 [==============================] - 2s 46ms/sample - loss: 2.5511e-07 - accuracy: 1.0000 - val_loss: 2.6970 - val_accuracy: 0.6200
Epoch 650/1000
50/50 [==============================] - 2s 46ms/sample - loss: 1.4305e-07 - accuracy: 1.0000 - val_loss: 2.6564 - val_accuracy: 0.6200
Epoch 651/1000
50/50 [==============================] - 2s 46ms/sample - loss: 2.2888e-07 - accuracy: 1.0000 - val_loss: 2.6423 - val_accuracy: 0.6200
Epoch 652/1000
50/50 [==============================] - 2s 47ms/sample - loss: 3.1233e-07 - accuracy: 1.0000 - val_loss: 2.6451 - val_accuracy: 0.6200
Epoch 653/1000
50/50 [==============================] - 2s 46ms/sample - loss: 2.0027e-07 - accuracy: 1.0000 - val_loss: 2.6627 - val_accuracy: 0.6200
Epoch 654/1000
50/50 [==============================] - 2s 46ms/sample - loss: 2.5749e-07 - accuracy: 1.0000 - val_loss: 2.6902 - val_accuracy: 0.6200
Epoch 655/1000
50/50 [==============================] - 2s 46ms/sample - loss: 2.7418e-07 - accuracy: 1.0000 

50/50 [==============================] - 2s 48ms/sample - loss: 1.7643e-07 - accuracy: 1.0000 - val_loss: 2.7141 - val_accuracy: 0.6200
Epoch 704/1000
50/50 [==============================] - 3s 58ms/sample - loss: 3.4571e-07 - accuracy: 1.0000 - val_loss: 2.6874 - val_accuracy: 0.6200
Epoch 705/1000
50/50 [==============================] - 2s 47ms/sample - loss: 2.4795e-07 - accuracy: 1.0000 - val_loss: 2.6918 - val_accuracy: 0.6200
Epoch 706/1000
50/50 [==============================] - 2s 46ms/sample - loss: 2.6703e-07 - accuracy: 1.0000 - val_loss: 2.7032 - val_accuracy: 0.6200
Epoch 707/1000
50/50 [==============================] - 2s 46ms/sample - loss: 1.2159e-07 - accuracy: 1.0000 - val_loss: 2.7151 - val_accuracy: 0.6200
Epoch 708/1000
50/50 [==============================] - 2s 46ms/sample - loss: 2.7418e-07 - accuracy: 1.0000 - val_loss: 2.7557 - val_accuracy: 0.6200
Epoch 709/1000
50/50 [==============================] - 2s 46ms/sample - loss: 1.9550e-07 - accuracy: 1.0000 